In [ ]:
project_name = "playground-series-s3e19"
token = {"username":"santanukundu","key":"de462df622bbe0817a45f17f3c1373eb"}

In [ ]:
import os, json
project_name = "playground-series-s3e19"
token = {"username":"santanukundu","key":"de462df622bbe0817a45f17f3c1373eb"}
os.environ['KAGGLE_CONFIG_DIR']='.'
with open('kaggle.json', "w") as f:
    json.dump(token, f)
!chmod 600 ./kaggle.json
!kaggle competitions download -c $project_name
project_name += ".zip"
!unzip $project_name && rm $project_name

  0% 0.00/1.18M [00:00<?, ?B/s]
100% 1.18M/1.18M [00:00<00:00, 115MB/s]
Archive:  playground-series-s3e19.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
import tensorflow as tf


In [ ]:
import holidays

In [ ]:
df =  pd.read_csv("train.csv")
df.head()

,id,date,country,store,product,num_sold
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49


In [ ]:
df.drop_duplicates(inplace=True)
df.drop('id',axis=1,inplace=True)
print("shape: ", df.shape)
df.isnull().sum()

shape:  (136950, 5)


date        0
country     0
store       0
product     0
num_sold    0
dtype: int64

In [ ]:
df.head()

,date,country,store,product,num_sold
0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49


In [ ]:
df['country'].unique()

array(['Argentina', 'Canada', 'Estonia', 'Japan', 'Spain'], dtype=object)

In [ ]:
df['store'].unique()

array(['Kaggle Learn', 'Kaggle Store', 'Kagglazon'], dtype=object)

In [ ]:
df['product'].unique()

array(['Using LLMs to Improve Your Coding',
       'Using LLMs to Train More LLMs',
       'Using LLMs to Win Friends and Influence People',
       'Using LLMs to Win More Kaggle Competitions',
       'Using LLMs to Write Better'], dtype=object)

In [ ]:
countries, stores, products = list(df['country'].unique()), list(df['store'].unique()), list(df['product'].unique())

In [ ]:
# i= 1
# for c in countries[:2]:
#   for s in stores[:2]:
#     for p in products[:2]:
#       x = df[(df['country']==c) & (df['store']==s) &(df['product']==p)][['date', 'num_sold']]
#       plt.figure(i, figsize=(20,10))
#       plt.plot(x['date'], x['num_sold'])
#       plt.ylabel("Num_of_Sales")
#       plt.xlabel("Date")
#       plt.title(f"country:{c}, store: {s}, product: {p}")
#       plt.show()

In [ ]:
c , s, p = 'Argentina', 'Kaggle Learn', 'Using LLMs to Improve Your Coding'
x = df[(df['country']==c) & (df['store']==s) &(df['product']==p)][['date', 'num_sold']]

In [ ]:
x.describe()

,num_sold
count,1826.000000
mean,29.885542
std,7.787840
min,12.000000
25%,24.000000
50%,29.000000
75%,34.000000
max,65.000000


In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler  =  MinMaxScaler()
input = np.array(x['num_sold']).reshape(-1,1)
x_train, x_test = input[:-31][:], input[-31:][:]
scaler.fit(input)
x_train = scaler.transform(x_train)
x_test =  scaler.transform(x_test)

In [ ]:
def preprocessing(df,col,split_method='fixed', p = -31, window_size = 100):
  if p<0 and split_method == 'ratio':
    print("p can not be negative!")
    return
  if split_method=='ratio' and (p>=1):
    print("Incorrect split!")
    return
  scaler = MinMaxScaler()
  _data = np.array(df[col]).reshape(-1,1)
  scaler.fit(_data)
  data = scaler.transform(_data).flatten()
  x = []
  y = []
  for i in range(0, len(data)-window_size):
    x.append(data[i:i+window_size])
    y.append([data[i+window_size]])
  x = np.asarray(x, dtype=object)
  y = np.asarray(y, dtype=object)
  if split_method == 'ratio':
    p = int(len(data)*p)
  x_train = np.asarray(x[:p][:]).astype(np.float32)
  y_train = np.asarray(y[:p][:]).astype(np.float32)
  x_test =  np.asarray(x[p:][:]).astype(np.float32)
  y_test =  np.asarray(y[p:][:]).astype(np.float32)
  return scaler, x_train, y_train, x_test, y_test


In [ ]:
from keras.api._v2.keras.layers import LSTM, Dense

def prediction(df, col, window_size=100):
  scaler,  x_train, y_train, x_test, y_test = preprocessing(df, col, window_size=window_size)
  conv_model = tf.keras.Sequential()
  conv_model.add(LSTM(50, return_sequences=True, input_shape=(100,1)))
  conv_model.add(LSTM(50))
  conv_model.add(Dense(1))
  conv_model.compile(loss='mean_squared_error', optimizer='adam')
  conv_model.summary()
  history = conv_model.fit(x_train, y_train, validation_data= (x_test,y_test), epochs=100, batch_size=64, verbose=1)

  import IPython
  x_ini = np.append(y_train.flatten(),y_test.flatten(), axis=0)[:100]

  for i in range(365):
    x_p = x_ini[:100].reshape(-1,100)
    x_ini=np.append(x_ini, conv_model.predict(x_p)[0])

  IPython.display.clear_output()
  y_pred = scaler.inverse_transform(x_ini[:365].reshape(-1,1)).flatten()
  return history, y_pred



In [ ]:
y_predict = np.asarray([])
for c in countries:
  for s in stores:
    for p in products:
      x = df[(df['country']==c) & (df['store']==s) &(df['product']==p)][['date', 'num_sold']]
      _, y_pred = prediction(x, 'num_sold')
      y_predict = np.append(y_predict, y_pred)

y_predict = y_predict.reshape(5,3,5,365)
y_reshaper = []
for i in range(365):
  for c in range(len(countries)):
    for s in range(len(stores)):
      for p in range(len(products)):
        y_reshaper.append(y_predict[c][s][p][i])


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_22 (LSTM)              (None, 100, 50)           10400     
                                                                 
 lstm_23 (LSTM)              (None, 50)                20200     
                                                                 
 dense_11 (Dense)            (None, 1)                 51        
                                                                 
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
27/27 [==============================] - 9s 163ms/step - loss: 0.0202 - val_loss: 0.0130
Epoch 2/100
27/27 [==============================] - 4s 133ms/step - loss: 0.0054 - val_loss: 0.0103
Epoch 3/100
27/27 [==============================] - 4s 160ms/step - loss: 0.0044 - val_loss: 0.0098
Epoch 4/10

In [ ]:
predict = pd.read_csv('test.csv')
predict['num_sold'] = y_reshaper
predict['num_sold'] = predict['num_sold'].map(lambda x: round(x))

In [ ]:
ans = predict[['id', 'num_sold']]
ans.to_csv('output.csv', index = False)

In [ ]:
pd.read_csv('test.csv')

In [ ]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [ ]:
# CONV_WIDTH = 3
# conv_model = tf.keras.Sequential([
#     tf.keras.layers.Conv1D(filters=32,
#                            kernel_size=(CONV_WIDTH,),
#                            activation='relu'),
#     tf.keras.layers.Dense(units=32, activation='relu'),
#     tf.keras.layers.Dense(units=1),
# ])

# CONV_WIDTH = 3
# conv_window = WindowGenerator(
#     input_width=CONV_WIDTH,
#     label_width=1,
#     shift=1,
#     label_columns=['T (degC)'])

# history = compile_and_fit(conv_model, conv_window)

# # IPython.display.clear_output()
# val_performance= {}
# performance = {}
# val_performance['Conv'] = conv_model.evaluate(conv_window.val)
# performance['Conv'] = conv_model.evaluate(conv_window.test, verbose=0)

In [ ]:
# def create_window(data, length=100):
#   x_train = np.asarray(data).flatten()
#   x = []
#   y = []
#   for i in range(0, len(x_train)-length):
#     x.append(x_train[i:i+length])
#     y.append(x_train[i+length])

#   return np.asarray(x), np.asarray(y).reshape(-1,1)

In [ ]:
# x_train, y_train = create_window(x_train)

In [ ]:
# test =  pd.read_csv("test.csv")
# test.head()

In [ ]:
# class WindowGenerator():
#   def __init__(self, input_width, label_width, shift,
#               #  train_df, val_df, test_df,
#               #  train_df=train_df, val_df=val_df, test_df=test_df,
#                label_columns=None):
#     # Store the raw data.
#     self.train_df = train_df
#     self.val_df = val_df
#     self.test_df = test_df

#     # Work out the label column indices.
#     self.label_columns = label_columns
#     if label_columns is not None:
#       self.label_columns_indices = {name: i for i, name in
#                                     enumerate(label_columns)}
#     self.column_indices = {name: i for i, name in
#                            enumerate(train_df.columns)}

#     # Work out the window parameters.
#     self.input_width = input_width
#     self.label_width = label_width
#     self.shift = shift

#     self.total_window_size = input_width + shift

#     self.input_slice = slice(0, input_width)
#     self.input_indices = np.arange(self.total_window_size)[self.input_slice]

#     self.label_start = self.total_window_size - self.label_width
#     self.labels_slice = slice(self.label_start, None)
#     self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

#   def __repr__(self):
#     return '\n'.join([
#         f'Total window size: {self.total_window_size}',
#         f'Input indices: {self.input_indices}',
#         f'Label indices: {self.label_indices}',
#         f'Label column name(s): {self.label_columns}'])

In [ ]:
# conv_window = WindowGenerator(
#     input_width=CONV_WIDTH,
#     label_width=1,
#     shift=1,
#     label_columns=['num_sold'])

# conv_window.val